In [1]:
#pip install tensorflow

  Created wheel for termcolor: filename=termcolor-1.1.0-cp37-none-any.whl size=4835 sha256=4b48a6064378da72cdcd13fc124d11bc37d644c009842f0857ca69ffd2700bb7
  Stored in directory: C:\Users\Arnold\AppData\Local\pip\Cache\wheels\7c\06\54\bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-none-any.whl size=19560 sha256=8494df7c6ecbd03a1d9756837ce60a82980b59c833c76cbdf3e88e791ef11ca5
  Stored in directory: C:\Users\Arnold\AppData\Local\pip\Cache\wheels\b1\c2\ed\d62208260edbd3fa7156545c00ef966f45f2063d0a84f8208a
Successfully built termcolor wrapt
  Found existing installation: six 1.12.0
    Uninstalling six-1.12.0:
      Successfully uninstalled six-1.12.0
  Found existing installation: numpy 1.16.5
    Uninstalling numpy-1.16.5:
      Successfully uninstalled numpy-1.16.5
  Found existing installation: wrapt 1.11.2
    Uninstalling wrapt-1.11.2:
      Successfully uninstalled wrapt-1.11.2
  Found existing installation: h5py 2.9.0
    Un

ERROR: astroid 2.3.1 has requirement six==1.12, but you'll have six 1.15.0 which is incompatible.
ERROR: astroid 2.3.1 has requirement wrapt==1.11.*, but you'll have wrapt 1.12.1 which is incompatible.


In [1]:
#for general
import pandas as pd
import numpy as np
import datetime as dt
import torch
#for image reading and preprocessing
import cv2
# for model building
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Activation, Add, AveragePooling2D, BatchNormalization, Conv2D, Dense, Dropout, Flatten, GlobalMaxPooling2D, Input, Lambda, MaxPooling2D, MaxPool2D, ZeroPadding2D

# for visualising images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load in data 

### First, load in the labels, and paths for image files

In [2]:
train = pd.read_csv("CheXpert-v1.0-small/train.csv")
train.columns = train.columns.str.lower().str.replace(" ","_").str.replace("/","_or_") #clean col names

In [3]:
train['cropped_path'] = train['path'].str.replace("CheXpert-v1.0-small/train/","cropped_equalized/")

C:\Users\Arnold\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


### Examine the distribution of labels in the dataset

In [4]:
#list of conditions
conditions = ['no_finding', 'enlarged_cardiomediastinum', 'cardiomegaly', 'lung_opacity',
              'lung_lesion', 'edema', 'consolidation', 'pneumonia', 
              'atelectasis', 'pneumothorax', 'pleural_effusion', 'pleural_other', 
              'fracture', 'support_devices']

#clean up the labels (blank for unmentioned, 0 for negative, -1 for uncertain, and 1 for positive)
#we convert unmentioned to negative
for col in train.columns:
    if col in conditions:
        train[col] = train[col].replace(np.nan,0)

#generate description of the data
train_desc = []

for col in train.columns:
    if col in conditions:
        if -1 in train[col].unique():
            train_desc.append([col,
                               str(train[col].groupby(train[col]).count()[1])+
                               ' ('+
                               str(round(train[col].groupby(train[col]).count()[1]/
                                         sum(train[col].groupby(train[col]).count())*100,2))+
                               '%)',
                               str(train[col].groupby(train[col]).count()[-1])+
                               ' ('+
                               str(round(train[col].groupby(train[col]).count()[-1]/
                                         sum(train[col].groupby(train[col]).count())*100,2))+
                               '%)',
                               str(train[col].groupby(train[col]).count()[0])+
                               ' ('+
                               str(round(train[col].groupby(train[col]).count()[0]/
                                         sum(train[col].groupby(train[col]).count())*100,2))+
                               '%)'])
        else:
            train_desc.append([col,
                               str(train[col].groupby(train[col]).count()[1])+
                               ' ('+
                               str(round(train[col].groupby(train[col]).count()[1]/
                                         sum(train[col].groupby(train[col]).count())*100,2))+
                               '%)',
                               0,
                               str(train[col].groupby(train[col]).count()[0])+
                               ' ('+
                               str(round(train[col].groupby(train[col]).count()[0]/
                                         sum(train[col].groupby(train[col]).count())*100,2))+
                               '%)'])

train_desc = pd.DataFrame(train_desc, columns = ['Pathology','Positive','Uncertain','Negative'])

In [5]:
sum(train[col].groupby(train[col]).count())

223414

In [6]:
train_desc

,Pathology,Positive,Uncertain,Negative
0,no_finding,22381 (10.02%),0,201033 (89.98%)
1,enlarged_cardiomediastinum,10798 (4.83%),12403 (5.55%),200213 (89.62%)
2,cardiomegaly,27000 (12.09%),8087 (3.62%),188327 (84.3%)
3,lung_opacity,105581 (47.26%),5598 (2.51%),112235 (50.24%)
4,lung_lesion,9186 (4.11%),1488 (0.67%),212740 (95.22%)
5,edema,52246 (23.39%),12984 (5.81%),158184 (70.8%)
6,consolidation,14783 (6.62%),27742 (12.42%),180889 (80.97%)
7,pneumonia,6039 (2.7%),18770 (8.4%),198605 (88.9%)
8,atelectasis,33376 (14.94%),33739 (15.1%),156299 (69.96%)
9,pneumothorax,19448 (8.7%),3145 (1.41%),200821 (89.89%)


### Extract a random subset of 1000 images to train toy model and another random subset of 300 for validation. Ensure that they don't overlap

In [7]:
#function to check training data is not in the validation set
def check_train(df, df2):
    for row in df.index:
        if row in list(df2.index):
            return False
        else:
            return True

In [8]:
#set a standard random seed for use for all random number generation
random_seed = 2021

#random sample for toy model
train_subset = train.sample(5000, random_state = random_seed)

#random sample for validation data
valid_subset = train.sample(300, random_state = 1000)
valid_subset['mask'] = check_train(valid_subset, train_subset)
valid_subset = valid_subset[valid_subset['mask']].drop(columns = 'mask')

In [9]:
#create train image set
train_data = []
for path in train_subset['cropped_path']:
    train_data.append(cv2.imread(path, cv2.IMREAD_UNCHANGED))

#create validation image set
valid_data = []
for path in valid_subset['cropped_path']:
    valid_data.append(cv2.imread(path, cv2.IMREAD_UNCHANGED))

In [10]:
#convert train_data to np array for the CNN input
train_data = np.array(train_data)

#convert valid_data to np array for the CNN input
valid_data = np.array(valid_data)

In [11]:
train_data

array([[[  1,   1,   1, ...,   1,   1,   2],
        [  1,   1,   1, ...,   6,   3,   3],
        [  1,   1,   1, ...,   3,   2,   0],
        ...,
        [145, 150, 144, ...,   6,   3,   7],
        [139, 157, 156, ...,   2,   0,   0],
        [150, 159, 153, ...,   3,   1,   5]],

       [[  6,   2,   5, ...,   5,   5,   5],
        [  4,   4,   0, ...,   5,   5,   5],
        [  1,   3,   2, ...,   5,   5,   5],
        ...,
        [214, 216, 217, ...,  53,  41,  46],
        [216, 217, 219, ...,  77,  78,  86],
        [215, 217, 218, ..., 121, 109, 113]],

       [[  7,   7,   7, ...,   7,   7,   7],
        [  7,   7,   7, ...,   7,   7,   7],
        [  7,   7,   7, ...,   7,   7,   7],
        ...,
        [  0,  37,  24, ...,   1,   2,   0],
        [  3,  25,  21, ...,   0,   0,   2],
        [  0,  23,  26, ...,   0,   2,   0]],

       ...,

       [[  7,   7,   8, ...,   6,   7,   8],
        [  8,   7,   6, ...,   4,   5,   5],
        [  7,   7,   8, ...,   4,   4,   4

In [12]:
# Normalize pixel values between -1 and 1
train_data = train_data/127.5 - 1
valid_data = valid_data/127.5 - 1
train_data

array([[[-0.99215686, -0.99215686, -0.99215686, ..., -0.99215686,
         -0.99215686, -0.98431373],
        [-0.99215686, -0.99215686, -0.99215686, ..., -0.95294118,
         -0.97647059, -0.97647059],
        [-0.99215686, -0.99215686, -0.99215686, ..., -0.97647059,
         -0.98431373, -1.        ],
        ...,
        [ 0.1372549 ,  0.17647059,  0.12941176, ..., -0.95294118,
         -0.97647059, -0.94509804],
        [ 0.09019608,  0.23137255,  0.22352941, ..., -0.98431373,
         -1.        , -1.        ],
        [ 0.17647059,  0.24705882,  0.2       , ..., -0.97647059,
         -0.99215686, -0.96078431]],

       [[-0.95294118, -0.98431373, -0.96078431, ..., -0.96078431,
         -0.96078431, -0.96078431],
        [-0.96862745, -0.96862745, -1.        , ..., -0.96078431,
         -0.96078431, -0.96078431],
        [-0.99215686, -0.97647059, -0.98431373, ..., -0.96078431,
         -0.96078431, -0.96078431],
        ...,
        [ 0.67843137,  0.69411765,  0.70196078, ..., -

### Prepare the train and validation labels

In [13]:
train_labels = np.array(train_subset[conditions])

valid_labels = np.array(valid_subset[conditions])

In [14]:
#decide how to treat the 'uncertain' label
#uncertain change to positive
train_labels_pos = np.array(train_subset[conditions].replace(-1,1))
valid_labels_pos = np.array(valid_subset[conditions].replace(-1,1))

#uncertain change to negative
train_labels_neg = np.array(train_subset[conditions].replace(-1,1))
valid_labels_neg = np.array(valid_subset[conditions].replace(-1,0))

# Set up and train the model

We shall build a multi-label classifier as opposed to a multi-output classifier. 

[NOTE] Should we consider trying both approaches? I'm worried that the multi-output classifier will take too long to train and tune as it is 14 models.

Credits for the pipeline go to https://towardsdatascience.com/building-a-multi-output-convolutional-neural-network-with-keras-ed24c7bc1178

### Define function to create multi-label model

In [15]:
def create_CNN_model(height,width,classes):

    #define input layer
    inputs = Input(shape = (height,width,1))

    #create the convolution/pooling layers
    #Conv2D -> BatchNormalization -> Pooling -> Dropout

    x = Conv2D(16, (3, 3), padding="same")(inputs)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(3, 3))(x)
    x = Dropout(0.25)(x)

    x = Conv2D(32, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    x = Conv2D(32, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(32, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(32, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    
    #create classification layers, first flatten the convolution output
    x = Flatten()(x)
    
    #create hidden layers for classification
    x = Dense(128)(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    #create output layer
    x = Dense(len(classes))(x)
    x = Activation("sigmoid", name = 'predicted_observations')(x) #sigmoid and not softmax because we are doing multi-label
    
    #create model class
    model = Model(inputs=inputs,
                  outputs = x)
    
    return model

### Define function to create ResNet152 model 

In [20]:
from keras.initializers import glorot_uniform

In [21]:
#define function to create identity blocks
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

In [22]:
#define function to create convolutional blocks
def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [23]:
#define function to create resnet model
def create_resnet50_model(height,width,classes):
    
    #define input layer
    inputs = Input(shape = (height, width, 1))
    
    #define resnet layers
    x = ZeroPadding2D((3, 3))(inputs)

    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(x)
    x = BatchNormalization(axis=3, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = convolutional_block(x, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')


    x = convolutional_block(x, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = convolutional_block(x, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = x = convolutional_block(x, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    x = AveragePooling2D(pool_size=(2, 2), padding='same')(x)
    
    #create classification layers, first flatten the convolution output
    x = Flatten()(x)
    
    #create hidden layers for classification
    x = Dense(128)(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    #create output layer
    x = Dense(len(classes))(x)
    x = Activation("sigmoid", name = 'predicted_observations')(x) #sigmoid and not softmax because we are doing multi-label
    
    model = Model(inputs=inputs, 
                  outputs=x, 
                  name='ResNet50')

    return model

### Define function to create DenseNet121 model

In [24]:
from keras.applications.densenet import DenseNet121

In [29]:
def create_densenet_model(height,width,classes):
    #define input layer
    inputs = Input(shape = (height,width,1))
    
    #create the ResNet152 layers
    x = DenseNet121(include_top = False,
                    input_tensor = inputs)
    
    #create classification layers, first flatten the convolution output
    x = Flatten()(x)
    
    #create hidden layers for classification
    x = Dense(128)(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    #create output layer
    x = Dense(len(classes))(x)
    x = Activation("sigmoid", name = 'predicted_observations')(x) #sigmoid and not softmax because we are doing multi-label
    
    #create model class
    model = Model(inputs=inputs,
                  outputs = x)
    
    return model

### Select the type of model and initialise

In [24]:
#initialise the model
model = create_resnet50_model(train_data.shape[1],train_data.shape[2],conditions)

### Define optimizer for the model

In [25]:
#need to do it in this specific way to access the keras API.
from keras.optimizers import adam_v2

In [26]:
init_lr = 1e-4
epochs = 5

opt = adam_v2.Adam(learning_rate=init_lr, decay=init_lr / epochs)

model.compile(optimizer = opt, 
              loss = tf.nn.sigmoid_cross_entropy_with_logits,
              metrics = [keras.metrics.AUC(from_logits = True, 
                                           multi_label = True)]
             )

### Train the model in batches

In [31]:
#use ModelCheckpoint to save the model to disk as we go along
from keras.callbacks import ModelCheckpoint
batch_size = 8
valid_batch_size = 12
callbacks = [
    ModelCheckpoint("./model_checkpoint")
]
history = model.fit(train_data,
                    train_labels_pos,
                    steps_per_epoch=len(train_data)//batch_size,
                    epochs=epochs,
                    callbacks = callbacks)

Epoch 1/5
625/625 [==============================] - 451s 687ms/step - loss: 0.8268 - auc: 0.5123


C:\Users\Arnold\Anaconda3\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: .\model_checkpoint\assets
Epoch 2/5
625/625 [==============================] - 438s 701ms/step - loss: 0.7796 - auc: 0.5212


C:\Users\Arnold\Anaconda3\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: .\model_checkpoint\assets
Epoch 3/5
625/625 [==============================] - 440s 704ms/step - loss: 0.7581 - auc: 0.5242


...

INFO:tensorflow:Assets written to: .\model_checkpoint\assets
Epoch 4/5
625/625 [==============================] - 456s 730ms/step - loss: 0.7320 - auc: 0.5263


C:\Users\Arnold\Anaconda3\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: .\model_checkpoint\assets
Epoch 5/5
625/625 [==============================] - 427s 684ms/step - loss: 0.7034 - auc: 0.5296


C:\Users\Arnold\Anaconda3\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: .\model_checkpoint\assets


# Visualise model performance 

In [21]:
y_pred = model.predict(valid_data)

In [32]:
model.evaluate(valid_data,
               valid_labels_pos)

10/10 [==============================] - 21s 787ms/step - loss: 0.6941 - auc: 0.5314


[0.6940694451332092, 0.5314294099807739]

In [ ]:
import plotly.graph_objects as go

In [ ]:
plt.clf()
fig = go.Figure()
fig.add_trace(go.Scatter(
                    y=history.history['predicted_observations_acc'],
                    name='Train'))
fig.add_trace(go.Scatter(
                    y=history.history['val_predicted_observations_acc'],
                    name='Valid'))
fig.update_layout(height=500, 
                  width=700,
                  title='Accuracy',
                  xaxis_title='Epoch',
                  yaxis_title='Accuracy')
fig.show()